---

**Note for running this code:** go to `Edit -> Notebook settings` and **pick T4 GPU** as the hardware accelerator.

---

# ME315 Day 6: Feedforward network (a.k.a Multilayer perceptron (MLP))

This code is structured to cover **binary and multi-class classification** using different MLP implementations.

It is based on curated tutorials and example datasets from TensorFlow.

Case studies:

1. Demo: **binary classification** of radar data
2. Activity: **Multiclass classification** of flower taxonomic data



---
## The 5-step model workflow

In previous examples, we have used **Keras** (the TensorFlow high-level API) to implement linear and logistic regression models, based on a standard workflow (or pipeline):

* Define the model
  * `model = ...`
* Compile the model
  * `model.compile (optimizer, loss_function, metrics)`
* Fit the model
  * `model.fit(X_training_features, y_training_labels, epochs, batch_size)`
* Evaluate the model
  * `loss = model.evaluate(X_testing_features, y_testing_labels)`
* Make predictions
  * `y_predicted = model.predict(X_new_data)`

---

## Step 0: software setup

We will concentrate all software installation and/or module import for these three examples in the next cell. In case of any errors, you can fix and/or install missing libraries before continuing.


In [ ]:
# importing necessary modules

# basic libraries
import numpy as np
from numpy import argmax
from numpy import sqrt
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython import display

# machine/deep learning
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense

# data loading and preprocessing
from pandas import read_csv
from sklearn.datasets import load_iris
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


To speed up training NNs, we use GPU (or could use TPU for Tensorflow). Access to a single GPU or TPU is available on Colab.
There is a restriction on usage - Colab do not provide details on what these restrictions are. If you reach the restriction, you won't have access to the GPU/TPU for a while- in which case you can switch back to the CPU.

In [ ]:

# Ensure that TensorFlow uses the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

---

# Demo: MLP for binary classification

This **binary classification** problem involves **determining whether a structure is in the atmosphere or not given radar returns**.

For that, we use the **[Ionosphere]((https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.names))** dataset, consisting of:

* 351 instances
* 34 attributes plus the class attribute
  * All 34 predictor attributes are continuous.
  * The 35th attribute is either "good" (g) or "bad" (b). *Good* radar returns are those showing evidence of some type of structure in the ionosphere. *Bad* returns are those that do not; their signals pass through the ionosphere.

The dataset will be downloaded automatically using Pandas, but you can learn more about it [here](https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv) (csv).



### 1.1 Data ingestion/acquisition

In [ ]:
# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
df = read_csv(path, header=None)

In [ ]:
df.head()

0   1        2        3        4        5        6        7        8   \
0   1   0  0.99539 -0.05889  0.85243  0.02306  0.83398 -0.37708  1.00000   
1   1   0  1.00000 -0.18829  0.93035 -0.36156 -0.10868 -0.93597  1.00000   
2   1   0  1.00000 -0.03365  1.00000  0.00485  1.00000 -0.12062  0.88965   
3   1   0  1.00000 -0.45161  1.00000  1.00000  0.71216 -1.00000  0.00000   
4   1   0  1.00000 -0.02401  0.94140  0.06531  0.92106 -0.23255  0.77152   

        9   ...       25       26       27       28       29       30  \
0  0.03760  ... -0.51171  0.41078 -0.46168  0.21266 -0.34090  0.42267   
1 -0.04549  ... -0.26569 -0.20468 -0.18401 -0.19040 -0.11593 -0.16626   
2  0.01198  ... -0.40220  0.58984 -0.22145  0.43100 -0.17365  0.60436   
3  0.00000  ...  0.90695  0.51613  1.00000  1.00000 -0.20099  0.25682   
4 -0.16399  ... -0.65158  0.13290 -0.53206  0.02431 -0.62197 -0.05707   

        31       32       33  34  
0 -0.54487  0.18641 -0.45300   g  
1 -0.06288 -0.13738 -0.02447   b  
2 -0.24180  0.56045 -0.38238   g  
3  1.00000 -0.32382  1.00000   b  
4 -0.59573 -0.04608 -0.65697   g  

[5 rows x 35 columns]

### 1.2 Data pre-processing

We need to separate input features (`X`) and labels/targets (`y`).

We also use a `LabelEncoder` to encode the class labels (strings) as integer values 0 and 1.

The model will be fit on 67 percent of the data (`X_train`, `y_train`), and the remaining 33 percent will be used for evaluation (`X_test`, `y_test`). We use the `train_test_split()` function for that.

In [ ]:
# split the dataset into input features (X) and class labels/targets (y)
X, y = df.values[:, :-1], df.values[:, -1]
# ensure all data are floating point values
X = X.astype('float32')
# encode strings to integer
y = LabelEncoder().fit_transform(y)
# split into training (67%) and testing (33%) samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training samples:", X_train.shape)
print("Training labels:", y_train.shape)
print("Testing samples:", X_test.shape)
print("Testing labels:", y_test.shape)

Training samples: (280, 34)
Training labels: (280,)
Testing samples: (71, 34)
Testing labels: (71,)


In [ ]:
# determine the number of input features
n_features = X_train.shape[1]
n_features

34

### 1.3 Model definition

We need to specify the architecture and parameters of our model.

For the architecture:
* the **shape of the input layer** (i.e., how many features will be used to train the model)
* **type and number** of hidden layers
* **how many nodes in each layer**
* **how many probabilities to output in the last layer**, as this is a binary classification problem.

For the parameters:
* which **activation function** to use in each layer
* whether to use any **kernel initialisation** function or not
* the **optimiser**
* which **loss function** to use

**Task:**
Build a MLP consisting of 3 layers:

- the first hidden layer has 10 units

- the second hidden layer has 8 units

- the output layer has 1 unit (as this is binary classification)

Use the ReLU activiation for the hidden layers, and the sigmoid function for the output.

Apply He initialisation to the two hidden layers

In [ ]:
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))



In [ ]:
# compile the model
# YOUR CODE HERE
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()




Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 10)                350       
                                                                 
 dense_9 (Dense)             (None, 8)                 88        
                                                                 
 dense_10 (Dense)            (None, 1)                 9         
                                                                 
Total params: 447 (1.75 KB)
Trainable params: 447 (1.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 1.4 Training the model

The model should be trained based on training input features and labels.

You need to specify: i) the iteration steps (`epochs`) and ii) the number of samples (`batches`) to be used during training.

In [ ]:
# fit the model


history = model.fit(X_train, y_train, validation_split=0.2, epochs=20, batch_size=32, verbose=1)
model.summary()

Epoch 1/20
7/7 - 0s - loss: 0.4673 - acc: 0.8750 - val_loss: 0.5128 - val_acc: 0.8571 - 202ms/epoch - 29ms/step
Epoch 2/20
7/7 - 0s - loss: 0.4559 - acc: 0.8750 - val_loss: 0.5046 - val_acc: 0.8571 - 79ms/epoch - 11ms/step
Epoch 3/20
7/7 - 0s - loss: 0.4451 - acc: 0.8750 - val_loss: 0.4969 - val_acc: 0.8571 - 78ms/epoch - 11ms/step
Epoch 4/20
7/7 - 0s - loss: 0.4345 - acc: 0.8750 - val_loss: 0.4891 - val_acc: 0.8393 - 79ms/epoch - 11ms/step
Epoch 5/20
7/7 - 0s - loss: 0.4244 - acc: 0.8750 - val_loss: 0.4817 - val_acc: 0.8393 - 82ms/epoch - 12ms/step
Epoch 6/20
7/7 - 0s - loss: 0.4144 - acc: 0.8795 - val_loss: 0.4739 - val_acc: 0.8393 - 62ms/epoch - 9ms/step
Epoch 7/20
7/7 - 0s - loss: 0.4048 - acc: 0.8795 - val_loss: 0.4658 - val_acc: 0.8571 - 76ms/epoch - 11ms/step
Epoch 8/20
7/7 - 0s - loss: 0.3951 - acc: 0.8929 - val_loss: 0.4586 - val_acc: 0.8571 - 74ms/epoch - 11ms/step
Epoch 9/20
7/7 - 0s - loss: 0.3859 - acc: 0.8973 - val_loss: 0.4511 - val_acc: 0.8750 - 78ms/epoch - 11ms/step
E

### 1.5: performance check (training)

We can plot the accuracy and loss values captured in the `history` object (callback) to analyze the performance of our model during the training phase.

In [ ]:
# visualising model's performance
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.arange(20), history.history['loss'], 'b-', label='loss')
ax.plot(np.arange(20), history.history['val_loss'], 'r-', label='val loss')
xlab, ylab = ax.set_xlabel('Epoch'), ax.set_ylabel('Loss')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.arange(20), history.history['acc'], 'b-', label='accuracy')
ax.plot(np.arange(20), history.history['val_acc'], 'r-', label='val accuracy')
xlab, ylab = ax.set_xlabel('Epoch'), ax.set_ylabel('Accuracy')
ax.legend()

### 1.6 Evaluating the model

For evaluation, we make use of testing input features and labels and usually keep track of `loss` and `accuracy` figures.

In [ ]:
# Evaluate the model

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)
print('    Test loss: %.3f' % loss)

Test Accuracy: 0.845
    Test loss: 0.381


### 1.7 Using the model for classification

For predicting a new class label, the model should be fed with some new (unseen) data instance.

In [ ]:
# make a prediction

#row is a single data point (there are 34 input variables)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = model.predict([row], verbose=0)
print('Predicted: %.3f' % yhat)

in `model.predict()`

`[row]`: This represents the input data for which you want to make a prediction. In this case, row is a single data instance, formatted as a list of features. The input is wrapped in another list ([row]) to create a batch with one sample because Keras models expect a batch of inputs.


### 1.8 Interpretability

In [ ]:
model.summary()

**Output Shape:**

`(None, 10)`: This specifies the shape of the output produced by this layer.
None: The first dimension (None) represents the batch size, which is flexible and not fixed during the model definition. It allows the model to handle variable batch sizes.
10: The second dimension (10) indicates that this layer outputs vectors of size 10. This means each input sample, when processed by this layer, will produce an output vector with 10 elements.

`Param #`:

350: This represents the total number of trainable parameters in this layer.

In [ ]:
# retrieving information on layers and corresponding weights
for layer in model.layers:
  print('Layer: ', layer)
  print('Weights: ', layer.get_weights())

---

# Activity: MLP for multiclass classification

We will use the **[Iris Plants](https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv)** dataset to demonstrate an MLP for **multiclass classification**. This problem involves **predicting the species of iris flower given measures of the flower**.

The dataset consists of:
* 150 instances (50 in each of three classes)
* Number of Attributes: 4 numeric, predictive attributes and the class
* Attribute Information:
   * sepal length in cm
   * sepal width in cm
   * petal length in cm
   * petal width in cm
   * class: Iris Setosa, Iris Versicolour, Iris Virginica

The dataset will be downloaded automatically using Scikit-learn, but you can learn more about it [here](https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.names).

In [ ]:
# uncomment the following lines to display an image with different types of Iris flowers
# REMEMBER to upload the iris-dataset.png into Google Colab
#from IPython.display import Image
#Image(filename='/content/w01_iris_dataset.png', width=600)

### 2.1 Data ingestion/acquisition

We will use the Iris dataset available in the [Scikit-learn](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html) library.

In [ ]:
# load the dataset from Scikit-learn
iris = load_iris()
print("Features:", iris.data[0])
print("Class:", iris.target[0])

### 2.2 Exploratory data analysis

We can plot the data to check how the classes relate to each other.

In [ ]:
# scatter plot of Iris data
# iris.data[:, 0]: sepal length
# iris.data[:, 1]: sepal width
# iris.data[:, 2]: petal length
# iris.data[:, 3]: petal width
_, ax = plt.subplots()
scatter = ax.scatter(iris.data[:, 0], iris.data[:, 1], c=iris.target)
ax.set(xlabel=iris.feature_names[0], ylabel=iris.feature_names[1])
_ = ax.legend(
    scatter.legend_elements()[0], iris.target_names, loc="lower right", title="Classes"
)

`[from documentation]`: Each point in the scatter plot refers to one of the 150 iris flowers in the dataset, with the color indicating their respective type (Setosa, Versicolour, and Virginica). You can already see a pattern regarding the Setosa type, which is easily identifiable based on its short and wide sepal. Only considering these 2 dimensions (`sepal width` and `sepal length`), there’s still overlap between the Versicolor and Virginica types.



### 2.3 Data pre-processing

We need to perform some pre-processing steps before submitting the data to our MLP model, as follows:

1. split the dataset into features and labels
2. converting all numerical values to the same data type
3. splitting features and labels into training and testing samples

Notice that the `LabelEncoder()` step is commented as the dataset already has all classes code as 0, 1 and 2; otherwise, we would need to apply this step as well.


In [ ]:
# split into input and output columns
X, y = iris.data, iris.target
# ensure all data are floating point values
X = X.astype('float32')
# encode strings to integer
#y = LabelEncoder().fit_transform(y)
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
print("X_train shape: %s - X_test shape: %s\ny_train shape: %s   - y_test shape: %s" % (X_train.shape, X_test.shape, y_train.shape, y_test.shape))
# determine the number of input features
n_features = X_train.shape[1]
print("\nNumber of features: ", n_features)

### 2.4 Model definition

We need to specify the architecture and parameters of your model.

For the architecture:
* the **shape of the input layer** (i.e., how many features will be used to train the model)
* **type and number** of hidden layers
* **how many probabilities to output in the last layer**, as this is a multiclass classification problem.

For the parameters:
* which **activation function** to use in each layer
* whether to use any **kernel initialisation** function or not
* the **optimiser**
* which **loss function** to use

**Activity: define model**
Using `Sequential()` build a MLP with the following ordered layers:

- 1st hidden layer: Dense with 10 units
- 2nd hidden layer: Dense with 8 units
- output layer:  Dense with 3 units

For the first and second hidden layers use ReLU activation, and He Normal initialisation.

For the output layer use the softmax activation.

Call your MLP `model`

In [ ]:
# Define model
# YOUR CODE HERE


**Activity: configure the model for training**

Compile your model with the following specifications:

- ADAM optimiser
- set loss to `sparse_categorical_crossentropy`. This loss function is used for multi-class classification problems where the target labels are integers (not one-hot encoded). It is useful when you have classes that are mutually exclusive.
- keep track of the accuracy metric

In [ ]:
# compile the model
# YOUR CODE HERE
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 2.5 Model training

For training the model, you can specific the **number of training epochs** and the **batch size** (number of data points) in each iteration.

**Activity**

Train the model with 20pc in the validation set, epoch to a low number like 3 to check it runs, batch size to 32.

In [ ]:
# fit the model
history = # YOUR CODE HERE

### 2.6: performance check (training)

We can plot the accuracy and loss values captured in the `history` object (callback) to analyze the performance of our model during the training phase.

In [ ]:
# visualising model's performance
# visualising model's performance
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.arange(100), history.history['loss'], 'b-', label='loss')
ax.plot(np.arange(100), history.history['val_loss'], 'r-', label='val loss')
xlab, ylab = ax.set_xlabel('Epoch'), ax.set_ylabel('Loss')
ax.legend()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.arange(100), history.history['accuracy'], 'b-', label='accuracy')
ax.plot(np.arange(100), history.history['val_accuracy'], 'r-', label='val acc')
xlab, ylab = ax.set_xlabel('Epoch'), ax.set_ylabel('Accuracy')
ax.legend()

### 2.7 Model evaluation

In [ ]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % acc)

### 2.6 Classification of new data

In [ ]:
# make a prediction
row = [5.1,3.5,1.4,0.2]
yhat = model.predict([row], verbose=0)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

### 2.7 Interpretability

In [ ]:
model.summary()

In [ ]:
# retrieving information on layers and corresponding weights
for layer in model.layers:
  print('Layer: ', layer)
  print('Weights: ', layer.get_weights())